In [1]:
import json
import os
from glob import glob

news_dir = '../data/newsdata'

news_list = []

for news_dir_i in glob(news_dir + '/*'):
    tmp_news_list = glob(news_dir_i + '/*')
    news_list += tmp_news_list
    
news_list = [x for x in news_list if x.endswith('.json')]
news_list.sort()

In [2]:
news_list

['../data/newsdata/news_1_1/NIRW2000000001.json',
 '../data/newsdata/news_1_1/NIRW2000000002.json',
 '../data/newsdata/news_1_1/NLRW2000000001.json',
 '../data/newsdata/news_1_1/NLRW2000000002.json',
 '../data/newsdata/news_1_1/NLRW2000000003.json',
 '../data/newsdata/news_1_1/NLRW2000000004.json',
 '../data/newsdata/news_1_1/NLRW2000000005.json',
 '../data/newsdata/news_1_1/NLRW2000000006.json',
 '../data/newsdata/news_1_1/NLRW2000000007.json',
 '../data/newsdata/news_1_1/NLRW2000000008.json',
 '../data/newsdata/news_1_1/NLRW2000000009.json',
 '../data/newsdata/news_1_1/NLRW2000000010.json',
 '../data/newsdata/news_1_1/NLRW2000000011.json',
 '../data/newsdata/news_1_1/NLRW2000000012.json',
 '../data/newsdata/news_1_1/NLRW2000000013.json',
 '../data/newsdata/news_1_1/NLRW2000000014.json',
 '../data/newsdata/news_1_1/NLRW2000000015.json',
 '../data/newsdata/news_1_1/NLRW2000000016.json',
 '../data/newsdata/news_1_1/NLRW2000000017.json',
 '../data/newsdata/news_1_1/NLRW2000000018.json',


In [6]:
import re
from kss import split_sentences

def preprocess(text):
    # 한 개의 평서문만 허용
    if text.count('.') != 1:
        return ''
    if '·' in text:
        return ''
    # \xa0
    text = re.sub('\xa0', ' ', text)

    # 괄호 안의 텍스트 제거
    text = re.sub(r'\([^)]*\)','', text)
    # <> 안의 텍스트 제거
    text = re.sub('<.+?>', '', text)
    # 큰따옴표 제거
    text = re.sub('"', '', text)
    # 다른 형태의 따옴표 처리
    text = re.sub(r'[‘’]', '', text)
    text = re.sub(r'[“”]', '', text)
    
    # 영어, 한국어를 제외한 다른 언어 제외
    if re.sub("[가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9.\W]", '', text):
        return ''
    
    # 너무 적은 개수의 토큰으로 이루어진 경우 제외(예: 어린 시절.)
    if len(text.split()) <= 5:
        return ''
    
    return text      

def news2sentlist(news_file):
    ret_list = []
    with open(news_file, 'r') as f:
        json_data = json.load(f)
    
    for i in range(len(json_data['document'])):
        tmp_data = json_data['document'][i]['paragraph']
    
        for j in range(len(tmp_data)):
            if j == 0: continue
            if len(tmp_data[j]['form']) <= 70:
                ret_list.append(tmp_data[j]['form'])
                
    ret_list = [preprocess(x) for x in ret_list if preprocess(x)]
    
    return ret_list

In [23]:
default_dir = '../data/newsdata/news_1_1/'
len(news2sentlist(default_dir + 'NPRW2000000001.json'))

19486

In [25]:
domain_sent = news2sentlist(default_dir + 'NPRW2000000001.json')

In [28]:
import pandas as pd

domain_df = pd.DataFrame(domain_sent)
domain_df.head()

,0
0,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다."
1,삼성전자 관계자는 지난해부터 중저가 모델에 멀티 카메라 등 프리미엄 기능을 먼저 넣...
2,SBS 신개념 음악 예능 더 팬이 단 한 회도 빠짐없이 2049 타깃 시청률 1위를...
3,"한편 최종 TOP5는 현장 관객 점수와 앱 투표 결과를 합산해 결정되며, 그 결과는..."
4,당권 도전을 기정사실화한 오 전 시장도 21일 영남권을 시작으로 지방 순회에 나선다.


In [29]:
domain_df.to_csv('../data/specific_domain_sentences.csv', index=False)


In [30]:
domain_sent[:5]

['삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.',
 '삼성전자 관계자는 지난해부터 중저가 모델에 멀티 카메라 등 프리미엄 기능을 먼저 넣고 있다고 말했다.',
 'SBS 신개념 음악 예능 더 팬이 단 한 회도 빠짐없이 2049 타깃 시청률 1위를 기록하고 있다.',
 '한편 최종 TOP5는 현장 관객 점수와 앱 투표 결과를 합산해 결정되며, 그 결과는 다음주 생방송 무대에서 공개된다.',
 '당권 도전을 기정사실화한 오 전 시장도 21일 영남권을 시작으로 지방 순회에 나선다.']

## entity 추출

In [31]:
from itertools import permutations
from pororo import Pororo

ner = Pororo(task='ner', lang='ko')


In [32]:
import requests

r = requests.get('https://raw.githubusercontent.com/datawhales/Korean_RE/main/data/relation/relid2label.json')
relid2label = json.loads(r.text)

In [33]:
list(relid2label.items())[:5]

[('P17', '다음 국가의 것임(country)'),
 ('P131', '다음 행정구역에 위치함(located in)'),
 ('P530', '수교국(diplomatic relation)'),
 ('P150', '하위 행정구역'),
 ('P47', '경계를 접하고 있음(shares border with)')]

In [34]:
relation_list = list(relid2label.keys())
relation_list[:5]

['P17', 'P131', 'P530', 'P150', 'P47']

In [35]:
def idx2relid(idx_list):
    label_out = []
    
    for idx in idx_list:
        label = relation_list[idx]
        label_out.append(label)
        
    return label_out

In [36]:
def ner_list(sentence):
    ner_result = ner(sentence)
    ner_result = [(item[0], item[1], len(item[0])) for item in ner_result]
    
    modified_list = []
    tmp_cnt = 0

    for item in ner_result:
        modified_list.append((item[0], item[1], [tmp_cnt, tmp_cnt + item[2]]))
        tmp_cnt += item[2]
    
    true_list = {'PERSON': True,'ORGANIZATION': True, 'COUNTRY': True, 'LOCATION': True, 'EVENT': True,
                 'THEORY': True, 'ARTIFACT': True, 'ANIMAL': True, 'TERM': True, 'STUDY_FIELD': True,
                 'MATERIAL': True, 'CITY': True, 'DISEASE': True}
    ent_list = [item for item in modified_list if item[1] in true_list]

    return ent_list

In [37]:
for i in range(len(domain_sent)):
    print(ner_list(domain_sent[i]))
    if i == 10: break

[('삼성전자', 'ORGANIZATION', [0, 4]), ('인도', 'COUNTRY', [6, 8]), ('중국', 'COUNTRY', [10, 12]), ('스마트폰', 'TERM', [34, 38])]
[('삼성전자', 'ORGANIZATION', [0, 4]), ('멀티 카메라', 'TERM', [24, 30])]
[('SBS', 'ORGANIZATION', [0, 3]), ('더 팬', 'ARTIFACT', [14, 17])]
[]
[('오', 'PERSON', [14, 15]), ('영남권', 'LOCATION', [26, 29])]
[('목포', 'LOCATION', [0, 2]), ('손혜원', 'PERSON', [14, 17]), ('더불어민주당', 'ORGANIZATION', [18, 24])]
[('손', 'PERSON', [0, 1])]
[('손', 'PERSON', [0, 1]), ('국회', 'ORGANIZATION', [18, 20])]
[('박', 'PERSON', [0, 1])]
[('북미정상회담', 'EVENT', [3, 9])]
[('스타벅스커피코리아', 'ORGANIZATION', [0, 9])]


In [54]:
import random

def get_all_entity_pairs(sentence):
    
    ent_list = ner_list(sentence)
    
#     if len(ent_list) >= 5:
        
#         return random.sample(list(permutations(ent_list, 2)), 3)
    
#     elif len(ent_list) <= 2:
#         return list(permutations(ent_list, 2))
    
#     return random.sample(list(permutations(ent_list, 2)), 3)
    return list(permutations(ent_list, 2))
    
def get_all_inputs_csv_form(sentence):
    pairs = get_all_entity_pairs(sentence)
    return [[sentence, sentence[ent_subj[2][0]:ent_subj[2][1]], ent_subj[2][0], 
             ent_subj[2][1], ent_subj[1], sentence[ent_obj[2][0]:ent_obj[2][1]],
             ent_obj[2][0], ent_obj[2][1], ent_obj[1]] for ent_subj, ent_obj in pairs]

In [55]:
get_all_entity_pairs(domain_sent[0])

[(('삼성전자', 'ORGANIZATION', [0, 4]), ('인도', 'COUNTRY', [6, 8])),
 (('삼성전자', 'ORGANIZATION', [0, 4]), ('중국', 'COUNTRY', [10, 12])),
 (('삼성전자', 'ORGANIZATION', [0, 4]), ('스마트폰', 'TERM', [34, 38])),
 (('인도', 'COUNTRY', [6, 8]), ('삼성전자', 'ORGANIZATION', [0, 4])),
 (('인도', 'COUNTRY', [6, 8]), ('중국', 'COUNTRY', [10, 12])),
 (('인도', 'COUNTRY', [6, 8]), ('스마트폰', 'TERM', [34, 38])),
 (('중국', 'COUNTRY', [10, 12]), ('삼성전자', 'ORGANIZATION', [0, 4])),
 (('중국', 'COUNTRY', [10, 12]), ('인도', 'COUNTRY', [6, 8])),
 (('중국', 'COUNTRY', [10, 12]), ('스마트폰', 'TERM', [34, 38])),
 (('스마트폰', 'TERM', [34, 38]), ('삼성전자', 'ORGANIZATION', [0, 4])),
 (('스마트폰', 'TERM', [34, 38]), ('인도', 'COUNTRY', [6, 8])),
 (('스마트폰', 'TERM', [34, 38]), ('중국', 'COUNTRY', [10, 12]))]

In [56]:
get_all_inputs_csv_form(domain_sent[0])

[['삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.',
  '삼성전자',
  0,
  4,
  'ORGANIZATION',
  '인도',
  6,
  8,
  'COUNTRY'],
 ['삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.',
  '삼성전자',
  0,
  4,
  'ORGANIZATION',
  '중국',
  10,
  12,
  'COUNTRY'],
 ['삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.',
  '삼성전자',
  0,
  4,
  'ORGANIZATION',
  '스마트폰',
  34,
  38,
  'TERM'],
 ['삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.',
  '인도',
  6,
  8,
  'COUNTRY',
  '삼성전자',
  0,
  4,
  'ORGANIZATION'],
 ['삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.',
  '인도',
  6,
  8,
  'COUNTRY',
  '중국',
  10,
  12,
  'COUNTRY'],
 ['삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.',
  '인도',
  6,
  8,
  'COUNTRY',
  '스마트폰',
  34,
  38,
  'TERM'],
 ['삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.',
  '중국',
  10,
  12,
  'COUNTRY',
  '삼성전자',
  0,
  4,
  'ORGANIZATION'],
 ['삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.',
  '중국',
  10,
  12,
  'COUNTRY',
  '인도',
  6,
  8,
  'COUNTRY'],
 ['삼성전자가 인도

In [62]:
from tqdm import tqdm

def get_news_test_data(sentences):
    ret_df = pd.DataFrame(columns=['sentence', 'subj_name', 'subj_start_pos', 'subj_end_pos',
                         'subj_type', 'obj_name', 'obj_start_pos', 'obj_end_pos', 'obj_type'])
    for i, sent in enumerate(tqdm(sentences)):
        tmp_df = pd.DataFrame(get_all_inputs_csv_form(sent),
                    columns=['sentence', 'subj_name', 'subj_start_pos', 'subj_end_pos',
                         'subj_type', 'obj_name', 'obj_start_pos', 'obj_end_pos', 'obj_type'])
        ret_df = pd.concat([ret_df, tmp_df], ignore_index=True)
#         if i == 10: break
    return ret_df

In [63]:
domain_test_df = get_news_test_data(domain_sent)

100%|██████████| 19486/19486 [40:45<00:00,  7.97it/s]


In [64]:
domain_test_df.head()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type
0,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.",삼성전자,0,4,ORGANIZATION,인도,6,8,COUNTRY
1,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.",삼성전자,0,4,ORGANIZATION,중국,10,12,COUNTRY
2,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.",삼성전자,0,4,ORGANIZATION,스마트폰,34,38,TERM
3,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.",인도,6,8,COUNTRY,삼성전자,0,4,ORGANIZATION
4,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.",인도,6,8,COUNTRY,중국,10,12,COUNTRY


In [65]:
domain_test_df.to_csv('../data/news_processed/specific_domain_test.csv', index=False)

In [1]:
import pandas as pd

new = pd.read_csv('../data/news_processed/specific_domain_test.csv')
new.head()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type
0,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.",삼성전자,0,4,ORGANIZATION,인도,6,8,COUNTRY
1,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.",삼성전자,0,4,ORGANIZATION,중국,10,12,COUNTRY
2,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.",삼성전자,0,4,ORGANIZATION,스마트폰,34,38,TERM
3,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.",인도,6,8,COUNTRY,삼성전자,0,4,ORGANIZATION
4,"삼성전자가 인도, 중국 시장을 겨냥해 혁신기술을 넣은 중저가 스마트폰을 연이어 선보인다.",인도,6,8,COUNTRY,중국,10,12,COUNTRY


In [3]:
new.tail()

,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type
70259,'더 뱅커' 김상중이 강단 있는 카리스마로 서이숙의 채용 비리 사건을 해결하고 비리...,꼬리,56,58,ANIMAL,김상중,7,10,PERSON
70260,'더 뱅커' 김상중이 강단 있는 카리스마로 서이숙의 채용 비리 사건을 해결하고 비리...,꼬리,56,58,ANIMAL,서이숙,24,27,PERSON
70261,'더 뱅커' 김상중이 강단 있는 카리스마로 서이숙의 채용 비리 사건을 해결하고 비리...,꼬리,56,58,ANIMAL,유동근,51,54,PERSON
70262,치료 대안이 마땅치 않은 말기암 환자들에게 희망을 줄 수 있는 혁신적인 항암바이러스...,말기암,14,17,TERM,항암바이러스,40,46,TERM
70263,치료 대안이 마땅치 않은 말기암 환자들에게 희망을 줄 수 있는 혁신적인 항암바이러스...,항암바이러스,40,46,TERM,말기암,14,17,TERM


In [8]:
new.sample(5)


,sentence,subj_name,subj_start_pos,subj_end_pos,subj_type,obj_name,obj_start_pos,obj_end_pos,obj_type
24727,현재 수감 중인 장대호는 일산동부경찰서를 나와 수사팀이 꾸려진 고양경찰서로 이동한다.,일산동부경찰서,14,21,ORGANIZATION,고양경찰서,35,40,ORGANIZATION
32480,8일 서울 강남구 임피리얼팰리스호텔에서 tvN 토일드라마 '호텔 델루나' 제작발표회...,tvN,22,25,ORGANIZATION,서울,3,5,CITY
26242,퍼퓸에서 하재숙과 조한철이 법원 앞에서 만나는 모습이 그려졌다.,퍼퓸,0,2,ARTIFACT,법원,15,17,ORGANIZATION
13698,에버랜드는 이와 함께 공식 페이스북 영상에 축하 맷글을 단 회원 10명을 선정해 1...,페이스북,15,19,ORGANIZATION,에버랜드,0,4,ORGANIZATION
62804,뮤지컬 웃는 남자가 국내 창작뮤지컬 최초로 영화관에서 정식 개봉한다.,뮤지컬 웃는 남자,0,9,ARTIFACT,창작뮤지컬,14,19,STUDY_FIELD


In [9]:
len(new)

70264